In [1]:
#include <limits.h>

unsigned long assignKey( char* some_line ){
    /*
       Function to convert the first 15 characters of a string
       into a numerical key to sort it by.
       
       If there are fewer then eight, it uses those.
    */
    char          *p;
    unsigned long  line_val, posn_val;
    int            i, use_chars;

    //scan along to the end of the string or for 8 chars, whichever comes first.
    p        = &some_line[0];
    for( i = 0; i < 15 && *p != '\0'; i++ ){ 
        p++;  //p++ "advance the pointer p".
    } 
    use_chars = i;
    p--; //now pointing to either the end of the line, or the eighth character.
    
    posn_val = 1;  //hundreds, tens, units etc but in base UCHAR_MAX (usually 256).
    line_val = 0;
    for( i = 8; i >= 0; i-- ){ //scan back again
        if( i < use_chars ){
            line_val += posn_val * ((unsigned char)*p--); //*p-- : "read value at p, then move p to the left".
        }
        posn_val *= UCHAR_MAX; //max value taken from limits.h
    } 
    
    return( line_val );
}

In [2]:
#include <stdio.h>     //input and output to file or screen
#include <stdlib.h>    //standard.... library...
#include<iostream>

int main(void)
{
    /*
        INITIALIZATION AND DEFINITION OF VARIABLES
    */
    FILE *f;
    int N = 50; // amount of lines to scan !!! DO NOT EXCEED 7, else the sorting algorithm explodes
    int i, j, k, line_count = 0, availableFlags;
    unsigned long *keyList;
    int *flags;
    char    *line_buf = NULL;  //this has to be intialised NULL: won't be allocated unless it is NULL
    size_t   line_buf_size = 0; //this variable is written to if zero, otherwise read.
    ssize_t  line_size; //"Signed Size_Type" : size of a buffer, but can be -1 if failed.
    int smallerKeys = 0, iterationNumber = 0;
    
    f = fopen("AssText.txt", "r"); // open the file in read mode
    
    for( i = 0; i < N; i++ )
    {
        //read a line, returning the number of chars in the line, not including the terminating NULL.
        //a buffer big enough to hold the line is allocated here: we pass a pointer-to-a-pointer as &line_buf
        //and we also pass the address of another size_t variable, letting us know the size of the buffer.
        line_size = getline(&line_buf, &line_buf_size, f);
        if( line_size <= 0 ){
            printf("Reached end of file, or otherwise failed to read. Returned: %i\n", (int)line_size);
            break; //exit the loop
        }

        printf("line %i \n", i+1);
        printf("Size: %i \n", (int)line_size); 
        printf("Buffer Size: %i\n", (int)line_buf_size);
        printf("Name: %s\n", line_buf);
        
        line_count += 1; // update the line amount
    }
    
    printf("there are %i lines \n\n", line_count);
    
    keyList = (unsigned long *)malloc(line_count*sizeof(unsigned long)); // allocate memory for the necessary amount of keys
    flags = (int *)malloc(line_count*sizeof(int));
    
    rewind(f); // return to the beginning of the file
    
    for( i = 0; i < line_count; i++ )
    {
        line_size = getline(&line_buf, &line_buf_size, f);
        
        if( line_size <= 0 ){
            printf("Reached end of file, or otherwise failed to read. Returned: %i\n", (int)line_size);
            break; //exit the loop
        }
        
        keyList[i] = assignKey(line_buf); // get a key for the name
        flags[i] = 0; // initialize the flags by assigning 0 for each flag
    }
    
    free(line_buf);
    line_buf = NULL;
    line_buf_size = 0;
    
    fclose(f); // close the file
    
    /*
        TEST TO SEE IF THE POINTER CONTAINS THE KEYS
    */
    for ( i = 0; i < line_count; i++)
    {
        printf("keyList[%i] = %lu \n", i, keyList[i]);
    }
    
    for ( i = 0; i < line_count; i++)
    {
        printf("flags[%i] = %i \n", i, flags[i]);
    }
    
    printf("\n");
    
    f = fopen( "sorted_key_list.txt", "w");
    
    /*
        SORTING ALGORITHM
    */  
    availableFlags = line_count;
    printf( "there are %i available flags\n\n", availableFlags);
    
    while ( availableFlags > 1)
    {
        for ( i = 0; i < line_count; i++)
        {
            //printf( "key[%i] = %lu has been selected\n", i, keyList[i]);
            
            if (flags[i] == 1)
            {
                //printf("the flag is not zero, skipping this key\n\n");
                continue;
            }
            
            else if ( flags[i] == 0)
            {
                //printf("the flag for the selected key is zero\n");
                
                for ( j = 0; j < line_count; j++)
                {
                    if ( flags[j] == 0 & flags[i] == 0)
                    { 
                        //printf("key[%i] is %lu \n", j, keyList[j]);
                        
                        if ( keyList[i] > keyList[j])
                        {
                            smallerKeys += 1; //printf("key[%i] is smaller than key[%i]\n", i, j);
                        }
                    }
                    else if ( flags[j] != 0)
                    {
                        //printf("flag[%i] is not zero! Skipping this key. \n", j);
                    }
                }
                if ( smallerKeys == 0)
                { 
                    fprintf(f, "%lu\n", keyList[i]);
                    flags[i] = 1;
                    availableFlags -=1;                                    
                }

            }
            //printf("there were %i smaller keys during this iteration\n", smallerKeys);
            //printf("there are %i available flags remaining\n\n", availableFlags);
            smallerKeys = 0; // reset the counter
        }
        
        iterationNumber += 1;
        
        if ( iterationNumber > 1e6) // failsafe
        {
            printf("forcefully exited after %i iterations\n", iterationNumber);
            break;
        }
    }
    
    for( i = 0; i < line_count; i++)
    {
        printf( "flag[%i] = %i \n", i, flags[i]);
    }
    
    for( i = 0; i < line_count; i++)
    {
        if ( flags[i] == 0)
        {
            fprintf(f, "%lu \n", keyList[i]);
        }
    }
    
    fclose(f);
    
    /*
        SAVE KEYS IN A SEPARATE .txt FILE
    */
    f = fopen("key_list.txt", "w"); // create or write the file
        
    for ( i = 0; i < line_count; i++) // standard loop to write each key in a new line into the file
    {
        fprintf( f, "%lu \n", keyList[i]);
    }
        
    fclose(f);

    free(keyList);
    free(flags);
}
main()

line 1 
Size: 6 
Buffer Size: 120
Name: Alfie

line 2 
Size: 6 
Buffer Size: 120
Name: Alice

line 3 
Size: 5 
Buffer Size: 120
Name: Alex

line 4 
Size: 12 
Buffer Size: 120
Name: Alejandrina

line 5 
Size: 6 
Buffer Size: 120
Name: Aleta

line 6 
Size: 6 
Buffer Size: 120
Name: Alena

line 7 
Size: 6 
Buffer Size: 120
Name: Aleen

line 8 
Size: 10 
Buffer Size: 120
Name: Alejandra

line 9 
Size: 10 
Buffer Size: 120
Name: Alexandra

line 10 
Size: 8 
Buffer Size: 120
Name: Alexine

line 11 
Size: 7 
Buffer Size: 120
Name: Alecia

line 12 
Size: 5 
Buffer Size: 120
Name: Alfy

line 13 
Size: 6 
Buffer Size: 120
Name: Alexa

line 14 
Size: 7 
Buffer Size: 120
Name: Alikee

line 15 
Size: 5 
Buffer Size: 120
Name: Alie

line 16 
Size: 7 
Buffer Size: 120
Name: Alexia

line 17 
Size: 7 
Buffer Size: 120
Name: Alicea

line 18 
Size: 6 
Buffer Size: 120
Name: Aline

line 19 
Size: 6 
Buffer Size: 120
Name: Alene

line 20 
Size: 6 
Buffer Size: 120
Name: Alina

line 21 
Size: 7 
Buffer Size

0

In [ ]:
// probier diesen Zauberausdruck mal in einer for schleife: strncpy(name[i], line_buf, strlen(line_buf));  